<a href="https://colab.research.google.com/github/utsav-195/fingerprint-recognition-using-siamese-network/blob/master/fingerprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
# importing the required libraries
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import cv2
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

from tensorflow import keras
from PIL import Image

In [2]:
def augment(folder,file):
  filename = folder + "/" + file
  # loading in the images
  image = imageio.imread(filename)

  flip_vr=iaa.Flipud(p=1.0)
  flip_vr_image= flip_vr.augment_image(image)

  save_filename = folder + "/" + file.split(".")[0] + "_flipped.png"
  cv2.imwrite(save_filename, flip_vr_image)

  rotate = iaa.Affine(rotate=(50, -50))
  rotated_image = rotate.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_rotated1.png"
  cv2.imwrite(save_filename, rotated_image)

  rotate = iaa.Affine(rotate=(50, -50))
  rotated_image = rotate.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_rotated2.png"
  cv2.imwrite(save_filename, rotated_image)

  crop = iaa.Crop(percent=(0, 0.3)) # crop image
  crop_image=crop.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_cropped1.png"
  cv2.imwrite(save_filename, crop_image)

  crop = iaa.Crop(percent=(0, 0.3)) # crop image
  crop_image=crop.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_cropped2.png"
  cv2.imwrite(save_filename, crop_image)

  contrast=iaa.GammaContrast(gamma=2.0)
  contrast_image =contrast.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_bright1.png"
  cv2.imwrite(save_filename, contrast_image)

  contrast=iaa.GammaContrast(gamma=1.4)
  contrast_image =contrast.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_bright2.png"
  cv2.imwrite(save_filename, contrast_image)

  blur = iaa.GaussianBlur(sigma=4.0)
  blur_image=blur.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_blur1.png"
  cv2.imwrite(save_filename, blur_image)

  blur = iaa.GaussianBlur(sigma=2.0)
  blur_image=blur.augment_image(image)
  save_filename = folder + "/" + file.split(".")[0] + "_blur2.png"
  cv2.imwrite(save_filename, blur_image)

In [7]:
dimen = 224

dir_path = "/content/drive/My Drive/fingerprint/images/"
out_path = "/content/drive/My Drive/fingerprint/processed_images/"
model_path = "/content/drive/My Drive/fingerprint/model/"

In [21]:
sub_dir_list = os.listdir(dir_path)

images = []
for i in range(len(sub_dir_list)):
  # image_names = os.listdir(os.path.join(dir_path, sub_dir_list[i]))
  # augment(os.path.join(dir_path, sub_dir_list[i]),image_names[0])
  image_names = os.listdir(os.path.join(dir_path, sub_dir_list[i]))
  sub_dir_images = []
  for image_path in image_names:
    path = os.path.join(dir_path, sub_dir_list[i], image_path )
    try:
      print(path)
      image = Image.open(path)
      resize_image = image.resize((dimen, dimen))
      array_ = list()
      for x in range(dimen):
        sub_array = list()
        for y in range(dimen):
          sub_array.append(resize_image.load()[x, y])
        array_.append(sub_array)
      image_data = np.array(array_)
      image = np.array(np.reshape(image_data, (dimen, dimen, 3))) / 255
      sub_dir_images.append(image)
      images.append(image)
    except:
      print('WARNING : File {} could not be processed.'.format(path))
  sub_dir_images = np.array(sub_dir_images)
  np.save( '{0}/{1}_processed.npy'.format(os.path.join(dir_path, sub_dir_list[i]),str(sub_dir_list[i])), sub_dir_images )
  print("Save Complete")

images = np.array(images)

/content/drive/My Drive/fingerprint/images/thumb/thumb.jpg
/content/drive/My Drive/fingerprint/images/thumb/thumb_flipped.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_rotated1.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_rotated2.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_cropped1.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_cropped2.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_bright1.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_bright2.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_blur1.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_blur2.png
/content/drive/My Drive/fingerprint/images/thumb/thumb_processed.npy
Save Complete
/content/drive/My Drive/fingerprint/images/first/first.jpg
/content/drive/My Drive/fingerprint/images/first/first_flipped.png
/content/drive/My Drive/fingerprint/images/first/first_rotated1.png
/content/drive/My Drive/fingerprint/images/first/first_rotate

In [ ]:
samples_1 = []
samples_2 = []
labels = []

for i in range(len(images)):
  for j in range(len(images)):
    samples_1.append(images[i])
    samples_2.append(images[j])
    t = i - i%10 +10
    if t - 10 <= j < t:
      labels.append(1)
    else:
      labels.append(0)

X1 = np.array(samples_1)
X2 = np.array(samples_2)
Y = np.array(labels)

np.save( '{}/images.npy'.format( out_path ), images )
np.save( '{}/x1.npy'.format( out_path ), X1 )
np.save( '{}/x2.npy'.format( out_path ), X2 )
np.save( '{}/y.npy'.format( out_path ) , Y )

In [ ]:
from tensorflow.keras import models , optimizers , losses ,activations , callbacks
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from PIL import Image
import tensorflow as tf
import time
import os
import numpy as np

class Recognizer (object) :

	def __init__( self ):

		# tf.logging.set_verbosity(tf.logging.ERROR)

		self.__DIMEN = 224

		input_shape = ((self.__DIMEN**2) * 3 , )
		convolution_shape = (self.__DIMEN , self.__DIMEN , 3)
		kernel_size_1 = (10 ,10)
		kernel_size_2 = (7 ,7)
		kernel_size_3 = (4 ,4)
		# kernel_size_2 = ( 7 , 7 )
	
		# pool_size_1 = ( 3 , 3 )
		# pool_size_2 = ( 2 , 2 )
		strides = 1

		seq_conv_model = [

			Reshape( input_shape=input_shape , target_shape=convolution_shape),

			Conv2D( 64, kernel_size=kernel_size_1 , strides=strides , activation='relu' ),
			# Conv2D( 32, kernel_size=kernel_size_1, strides=strides, activation='relu'),
			MaxPooling2D(),

			Conv2D( 128, kernel_size=kernel_size_2 , strides=strides , activation='relu'),
			# Conv2D( 64, kernel_size=kernel_size_2 , strides=strides , activation='relu'),
			MaxPooling2D(),

      Conv2D( 128, kernel_size=kernel_size_3 , strides=strides , activation='relu'),
			# Conv2D( 128, kernel_size=kernel_size_3 , strides=strides , activation='relu'),
			MaxPooling2D(),

			Conv2D( 256, kernel_size=kernel_size_3 , strides=strides , activation='relu'),
			# Conv2D( 128, kernel_size=kernel_size_3 , strides=strides , activation='relu'),

			Flatten(),

			Dense(1024, activation='relu'),
	 		Dense(1024, activation=activations.sigmoid)

		]

		seq_model = tf.keras.Sequential(seq_conv_model)

		input_x1 = Input(shape=input_shape)
		input_x2 = Input(shape=input_shape)

		output_x1 = seq_model(input_x1)
		output_x2 = seq_model(input_x2)

		distance_euclid = Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] ))( [output_x1 , output_x2] )
		outputs = Dense( 1 , activation=activations.sigmoid) ( distance_euclid )
		self.__model = models.Model( [ input_x1 , input_x2 ] , outputs )

		self.__model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.00003))



	def fit(self, X, Y, hyperparameters):
		initial_time = time.time()
		self.__model.fit( X, Y,
						 batch_size=hyperparameters[ 'batch_size' ] ,
						 epochs=hyperparameters[ 'epochs' ] ,
						 callbacks=hyperparameters[ 'callbacks'],
						 validation_data=hyperparameters[ 'val_data' ]
						 )
		final_time = time.time()
		eta = (final_time - initial_time)
		time_unit = 'seconds'
		if eta >= 60 :
			eta = eta / 60
			time_unit = 'minutes'
		self.__model.summary( )
		print( 'Elapsed time acquired for {} epoch(s) -> {} {}'.format( hyperparameters[ 'epochs' ] , eta , time_unit ) )


	def prepare_images_from_dir( self , dir_path , flatten=True ):
		images_names = os.listdir(dir_path)
		f = [name for name in images_names if '.npy' in name]
		if len(f) == 0:
			images = list()
			for imageName in images_names:
				# print(imageName)
				image = Image.open(dir_path + imageName)
				resize_image = image.resize((self.__DIMEN, self.__DIMEN))
				array = list()
				for x in range(self.__DIMEN):
					sub_array = list()
					for y in range(self.__DIMEN):
						sub_array.append(resize_image.load()[x, y])
					array.append(sub_array)
				image_data = np.array(array)
				image = np.array(np.reshape(image_data,(self.__DIMEN, self.__DIMEN, 3))) /255
				images.append(image)

			if flatten:
				images = np.array(images)
				return images.reshape(images.shape[0], self.__DIMEN**2 * 3)).astype(np.float32)
			else:
				return np.array(images)
		else:
			images = np.load('{0}{1}'.format(dir_path,f[0]))
			if flatten:
				images = np.array(images)
				return images.reshape((images.shape[0], self.__DIMEN**2 * 3)).astype(np.float32)
			else:
				return np.array(images)


	def evaluate(self, test_X, test_Y) :
		return self.__model.evaluate(test_X, test_Y)


	def predict(self, X):
		predictions = self.__model.predict(X)
		return predictions


	def summary(self):
		self.__model.summary()

	def save_model(self, file_path):
		self.__model.save(file_path)


	def load_model(self, file_path):
		self.__model = models.load_model(file_path)

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from PIL import Image
import numpy as np
import time

data_dimension = 224

X1 = np.load('{}/x1.npy'.format(out_path))
X2 = np.load('{}/x2.npy'.format(out_path))
Y = np.load('{}/y.npy'.format(out_path))

X1 = X1.reshape((X1.shape[0], data_dimension**2 * 3)).astype(np.float32)
X2 = X2.reshape((X2.shape[0], data_dimension**2 * 3)).astype(np.float32)

print(X1.shape)
print(X2.shape)
print(Y.shape)

recognizer = Recognizer()

from keras.callbacks import ModelCheckpoint

# checkpointer to save the best model
checkpointer = ModelCheckpoint(filepath='{}/model_test5.h5'.format(model_path), monitor='loss', verbose=1, save_best_only=True)

parameters = {
    'batch_size' : 50,
    'epochs' : 30,
    'callbacks' : checkpointer,
    'val_data' : None
}

recognizer.fit( [ X1 , X2 ], Y, hyperparameters=parameters)

(2500, 150528)
(2500, 150528)
(2500,)
Epoch 1/20
50/50 [==============================] - ETA: 0s - loss: 0.5580
Epoch 00001: loss improved from inf to 0.55796, saving model to /content/drive/My Drive/fingerprint/model/model_test4.h5
50/50 [==============================] - 30s 603ms/step - loss: 0.5580
Epoch 2/20
50/50 [==============================] - ETA: 0s - loss: 0.4466
Epoch 00002: loss improved from 0.55796 to 0.44657, saving model to /content/drive/My Drive/fingerprint/model/model_test4.h5
50/50 [==============================] - 33s 655ms/step - loss: 0.4466
Epoch 3/20
50/50 [==============================] - ETA: 0s - loss: 0.3916
Epoch 00003: loss improved from 0.44657 to 0.39160, saving model to /content/drive/My Drive/fingerprint/model/model_test4.h5
50/50 [==============================] - 33s 662ms/step - loss: 0.3916
Epoch 4/20
50/50 [==============================] - ETA: 0s - loss: 0.3313
Epoch 00004: loss improved from 0.39160 to 0.33134, saving model to /content/d

In [ ]:
recognizer = Recognizer()
recognizer.load_model('{}/model_test5.h5'.format(model_path))

In [ ]:
test_images = recognizer.prepare_images_from_dir("/content/drive/My Drive/fingerprint/test_images/")
class_images = "/content/drive/My Drive/fingerprint/images/"

samples = {}

for class_name in os.listdir(class_images):
  samples[class_name] = recognizer.prepare_images_from_dir(class_images + class_name + "/")

test_images_names = os.listdir("/content/drive/My Drive/fingerprint/test_images/")

i = 0
threshold = 0.9

for image in test_images:
  image = image.reshape((1, -1))
  for class_name in os.listdir(class_images):
    for sample in samples[class_name][:2]:
      # print(class_name)
      sample = sample.reshape((1 ,-1))
      prediction_score = recognizer.predict([image, sample])[0]
      # print(prediction_score)
      if prediction_score > threshold:
          print( 'IMAGE {} is {} with confidence of {}'.format( test_images_names[i]  , class_name, prediction_score[0]) )
          break
    if prediction_score > threshold:
      break
  i += 1

IMAGE thumb.jpg is thumb with confidence of 0.9918048977851868
IMAGE first.jpg is first with confidence of 0.9564779996871948
IMAGE middle.jpg is middle with confidence of 0.9842307567596436
IMAGE pinky.jpg is pinky with confidence of 0.9520330429077148
IMAGE ring.jpg is ring with confidence of 0.9732805490493774
IMAGE thumb_test.jpg is thumb with confidence of 0.9593909978866577
IMAGE pinky_test.jpg is pinky with confidence of 0.9787534475326538


## Adding a new peron's fingerprint

In [ ]:
# adding new fingerprint steps
1. augment
2. preprocess
3. add to X1, X2 Y npy
4. retrain model
5. save model
6. test

In [3]:
# augment
folder = "/content/drive/My Drive/fingerprint/images/thumb_copy"
file_name = "thumb.jpg"
augment(folder,file_name)

In [25]:
#pre-process
dimen = 224
image_names = os.listdir(folder)

images = []
for image_path in image_names:
  path = os.path.join(folder, image_path)
  try:
    print(path)
    image = Image.open(path)
    resize_image = image.resize((dimen, dimen))
    array_ = list()
    for x in range(dimen):
      sub_array = list()
      for y in range(dimen):
        sub_array.append(resize_image.load()[x, y])
      array_.append(sub_array)
    image_data = np.array(array_)
    image = np.array(np.reshape(image_data, (dimen, dimen, 3))) / 255
    images.append(image)
  except:
      print('WARNING : File {} could not be processed.'.format(path))

images = np.array(images)
np.save('{0}/{1}_processed.npy'.format(folder,folder.split("/")[-1]), images)
print("Save Complete")

/content/drive/My Drive/fingerprint/images/thumb_copy/thumb.jpg
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_flipped.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_rotated1.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_rotated2.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_cropped1.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_cropped2.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_bright1.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_bright2.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_blur1.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_blur2.png
/content/drive/My Drive/fingerprint/images/thumb_copy/thumb_copy_processed.npy
Save Complete


In [26]:
# add to X1, X2 Y npy
saved_images = np.load('{}/images.npy'.format(out_path))
print("Shape of saved images")
print(saved_images.shape)
new_images = np.concatenate((saved_images,images))
print("Shape of new images")
print(new_images.shape)
np.save( '{}/images.npy'.format(out_path), new_images)

Shape of saved images
(50, 224, 224, 3)
Shape of new images
(60, 224, 224, 3)


In [28]:
samples_1 = []
samples_2 = []
labels = []

for i in range(len(new_images)):
  for j in range(len(new_images)):
    samples_1.append(new_images[i])
    samples_2.append(new_images[j])
    t = i - i%10 +10
    if t - 10 <= j < t:
      labels.append(1)
    else:
      labels.append(0)

X1 = np.array(samples_1)
X2 = np.array(samples_2)
Y = np.array(labels)

np.save( '{}/x1.npy'.format( out_path ), X1 )
np.save( '{}/x2.npy'.format( out_path ), X2 )
np.save( '{}/y.npy'.format( out_path ) , Y )

In [ ]:
# retrain model
use previous code
# save model
use previous code
# test
use previous code